# ДЗ Урок 6

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt
from itertools import combinations

plt.style.use('seaborn-whitegrid')

In [2]:
def is_concordant(pair1: tuple, pair2: tuple) -> bool:
    """Являются ли две пары согласованными.
    """
    
    return (pair1[0] - pair2[0]) * (pair1[1] - pair2[1]) > 0

## Задача 1

Даны значения величины заработной платы заемщиков банка (salary) и значения их поведенческого кредитного скоринга (scoring):

salary = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110]

scoring = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]

Используя только встроенные питоновские функции и структуры данных (т.е. без библиотек numpy, pandas и др.) найдите:
1. ковариацию (смещённую и несмещённую) этих двух величин,
2. коэффициент корреляции Пирсона,
3. коэффициент корреляции Кендалла.

Можно затем посчитать те же значения с использованием библиотек, чтобы проверить себя.

In [3]:
salary = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110]
salary_s = pd.Series([35, 45, 190, 200, 40, 70, 54, 150, 120, 110])
scoring = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]
scoring_s = pd.Series([401, 574, 874, 919, 459, 739, 653, 902, 746, 832])

In [4]:
salary_mean = sum(salary) / len(salary)
salary_mean

101.4

In [5]:
scoring_mean = sum(scoring) / len(scoring)
scoring_mean

709.9

In [6]:
salary_list = []
for i in salary:
    salary_list.append(i - salary_mean)
salary_list

[-66.4,
 -56.400000000000006,
 88.6,
 98.6,
 -61.400000000000006,
 -31.400000000000006,
 -47.400000000000006,
 48.599999999999994,
 18.599999999999994,
 8.599999999999994]

In [7]:
scoring_list = []
for i in scoring:
    scoring_list.append(i - scoring_mean)
scoring_list

[-308.9,
 -135.89999999999998,
 164.10000000000002,
 209.10000000000002,
 -250.89999999999998,
 29.100000000000023,
 -56.89999999999998,
 192.10000000000002,
 36.10000000000002,
 122.10000000000002]

In [8]:
item_sum = 0
for i in range(10):
    item = salary_list[i] * scoring_list[i]
    item_sum = item_sum + item
item_sum

91578.4

### Ковариация

In [9]:
cov_s = item_sum / len(salary)
cov_n = item_sum / (len(salary) - 1)
print(f'Смещенная ковариация {cov_s}')
print(f'Несмещенная ковариация {cov_n}')

Смещенная ковариация 9157.84
Несмещенная ковариация 10175.377777777778


In [10]:
np.cov(salary_s, scoring_s, ddof=1)

array([[ 3882.93333333, 10175.37777778],
       [10175.37777778, 33854.32222222]])

In [11]:
salary_n_o_d = 0
for i in salary:
    salary_n_o_d = salary_n_o_d + (i - salary_mean) ** 2 / (len(salary) - 1)
salary_n_o_d

3882.933333333334

In [12]:
scoring_n_o_d = 0
for i in scoring:
    scoring_n_o_d = scoring_n_o_d + (i - scoring_mean) ** 2 / (len(scoring) - 1)
scoring_n_o_d

33854.32222222222

In [13]:
salary_std = salary_n_o_d ** 0.5
salary_std

62.31318747531163

In [14]:
scoring_std = scoring_n_o_d ** 0.5
scoring_std

183.99544076477062

### коэффициент корреляции Пирсона

In [15]:
corr = cov_n / (salary_std * scoring_std)
corr

0.8874900920739163

In [16]:
np.corrcoef(salary_s, scoring_s)

array([[1.        , 0.88749009],
       [0.88749009, 1.        ]])

### коэффициент корреляции Кендалла

In [17]:
P = 0
Q = 0

for pair1, pair2 in combinations(zip(salary, scoring), r=2):
    if is_concordant(pair1, pair2):
        P += 1
    else:
        Q += 1
        
P, Q

(43, 2)

In [18]:
tau = (P - Q) / (P + Q)
tau

0.9111111111111111

In [19]:
pd.DataFrame({'salary': salary_s, 'scoring': scoring_s}).corr(method='kendall')

,salary,scoring
salary,1.000000,0.911111
scoring,0.911111,1.000000


## Задача 2

Измерены значения IQ выборки студентов, обучающихся в местных технических вузах:

131, 125, 115, 122, 131, 115, 107, 99, 125, 111

Известно, что в генеральной совокупности IQ распределен нормально. Найдите доверительный интервал для математического ожидания с надежностью 0.95.

Дисперсия неизвестна, поэтому используем распределение Стьюдента

In [20]:
IQ = np.array([131, 125, 115, 122, 131, 115, 107, 99, 125, 111])

In [21]:
n = IQ.shape[0]
mean = IQ.mean()
std = IQ.std(ddof=1)

n, mean, std

(10, 118.1, 10.54566788359614)

In [22]:
p = 0.95
alpha = 1 - p
alpha

0.050000000000000044

In [23]:
t = stats.t.ppf(1 - alpha / 2, df=n - 1)
t

2.2621571627409915

In [24]:
(mean - t * std / n**0.5, mean + t * std / n**0.5)

(110.55608365158724, 125.64391634841274)

## Задача 3

Известно, что рост футболистов в сборной распределён нормально с известной дисперсией 25. На выборке объёма 27 выборочное среднее составило 174.2. Найдите доверительный интервал для математического ожидания с надёжностью 0.95.

Дисперсия известна, поэтому используем нормальное распределение

In [25]:
d = 25
std = 5
n = 27
mean = 174.2
p = 0.95
alpha = 0.05

In [26]:
t = stats.norm.ppf(1 - alpha / 2)
t

1.959963984540054

In [27]:
(mean - t * std / n**0.5, mean + t * std / n**0.5)

(172.3140237765397, 176.08597622346028)